# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0594ee1610>
├── 'a' --> tensor([[ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333]])
└── 'x' --> <FastTreeValue 0x7f0594ee15b0>
    └── 'c' --> tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                        [-0.7719, -0.3942,  1.6017, -0.2252],
                        [-0.6502,  1.8978,  1.6624, -0.3462]])

In [4]:
t.a

tensor([[ 1.1110,  1.3867,  1.1788],
        [-0.8983,  0.8259,  0.0333]])

In [5]:
%timeit t.a

76.9 ns ± 0.74 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0594ee1610>
├── 'a' --> tensor([[ 0.6401,  0.4358, -1.5723],
│                   [-0.1610, -0.4269, -0.7177]])
└── 'x' --> <FastTreeValue 0x7f0594ee15b0>
    └── 'c' --> tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                        [-0.7719, -0.3942,  1.6017, -0.2252],
                        [-0.6502,  1.8978,  1.6624, -0.3462]])

In [7]:
%timeit t.a = new_value

90.7 ns ± 1.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333]]),
    x: Batch(
           c: tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                      [-0.7719, -0.3942,  1.6017, -0.2252],
                      [-0.6502,  1.8978,  1.6624, -0.3462]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1110,  1.3867,  1.1788],
        [-0.8983,  0.8259,  0.0333]])

In [11]:
%timeit b.a

69.8 ns ± 0.67 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1949,  0.0853, -0.4415],
               [-1.7424, -0.3308,  1.0565]]),
    x: Batch(
           c: tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                      [-0.7719, -0.3942,  1.6017, -0.2252],
                      [-0.6502,  1.8978,  1.6624, -0.3462]]),
       ),
)

In [13]:
%timeit b.a = new_value

631 ns ± 8.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 6.65 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.7 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

297 µs ± 7.31 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

278 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f04e8bd9fa0>
├── 'a' --> tensor([[[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]],
│           
│                   [[ 1.1110,  1.3867,  1.1788],
│                    [-0.8983,  0.8259,  0.0333]]])
└── 'x' --> <FastTreeValue 0x7f04e8bd9fd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.3 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f04e8bbd610>
├── 'a' --> tensor([[ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333],
│                   [ 1.1110,  1.3867,  1.1788],
│                   [-0.8983,  0.8259,  0.0333]])
└── 'x' --> <FastTreeValue 0x7f04e8bbdbb0>
    └── 'c' --> tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                        [-0.7719, -0.3942,  1.6017, -0.2252],
                 

In [23]:
%timeit t_cat(trees)

44.1 µs ± 1.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

80.9 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]],
       
               [[ 1.1110,  1.3867,  1.1788],
                [-0.8983,  0.8259,  0.0333]]]),
    x: Batch(
           c: tensor([[[-0.6396, -1.6913,  0.1114,  1.2212],
                       [-0.7719, -0.3942,  1.6017, -0.2252],
                       [-0.6502,  1.8978,  1.6624, -0.3462]],
         

In [26]:
%timeit Batch.stack(batches)

104 µs ± 1.84 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333],
               [ 1.1110,  1.3867,  1.1788],
               [-0.8983,  0.8259,  0.0333]]),
    x: Batch(
           c: tensor([[-0.6396, -1.6913,  0.1114,  1.2212],
                      [-0.7719, -0.3942,  1.6017, -0.2252],
                      [-0.6502,  1.8978,  1.6624, -0.3462],
                      [-0.6396, -1.6913,  0.1114,  1.2212],
                      [-0.7719, -

In [28]:
%timeit Batch.cat(batches)

185 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

588 µs ± 76.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
